# Cómo crear una aplicación Retriever LLM sencilla con LangChain
* Aplicación Retriever LLM muy sencilla sobre una fuente de datos de texto.
* Las aplicaciones Retriever pueden responder preguntas sobre documentos específicos.

In [ ]:
import warnings
import os

from langchain._api import LangChainDeprecationWarning
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']

# Creamos el modelo a utilizar
chatModel = ChatOpenAI(model="gpt-4o-mini")

# Creamos nuestro formateador de salida
output_parser = StrOutputParser()

### Instalar base de datos Chroma

Vamos a instalar una base de datos vectorial con el comando `poetry add langchain-chroma`

## Documentos
* Un documento LangChain está pensado para almacenar texto y metadatos.
* Tiene 2 atributos:
    * `page_content`
    * `metadata`

In [ ]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="John F. Kennedy served as the 35th president of the United States from 1961 until his assassination in 1963.",
        metadata={"source": "us-presidents-doc"},
    ),
    Document(
        page_content="Robert F. Kennedy was a key political figure and served as the U.S. Attorney General; he was also assassinated in 1968.",
        metadata={"source": "us-politics-doc"},
    ),
    Document(
        page_content="The Kennedy family is known for their significant influence in American politics and their extensive philanthropic efforts.",
        metadata={"source": "kennedy-family-doc"},
    ),
    Document(
        page_content="Edward M. Kennedy, often known as Ted Kennedy, was a U.S. Senator who played a major role in American legislation over several decades.",
        metadata={"source": "us-senators-doc"},
    ),
    Document(
        page_content="Jacqueline Kennedy Onassis, wife of John F. Kennedy, was an iconic First Lady known for her style, poise, and dedication to cultural and historical preservation.",
        metadata={"source": "first-lady-doc"},
    ),
]

### Base de datos Vectorial vs Retriever

Vamos a destrabar la diferencia entre una `Vector Store` y un `Retriever`.

#### Vector Store
Piense en un `Vector Store` como un espacio de almacenamiento especializado donde la información se guarda en un formato muy específico:
- **Almacenamiento de vectores**: un vector store guarda la información como vectores. Estos vectores son representaciones numéricas de texto, lo que facilita que las máquinas comprendan y comparen la información rápidamente.
- **Propósito**: el objetivo principal de un vector store es almacenar y recuperar de manera eficiente estos vectores. Cuando necesita encontrar qué tan similares son dos piezas de información, el vector store ayuda comparando rápidamente sus vectores.
- **Uso**: son cruciales en sistemas donde necesita realizar búsquedas de similitud en grandes conjuntos de datos. Por ejemplo, encontrar documentos que traten temas similares o identificar consultas de usuarios similares.

#### Retrievers
Por otro lado, los retrievers se centran más en la búsqueda activa de información:
- **Recuperación de información**: un retriever toma una consulta (como una pregunta o un término de búsqueda) y busca en una base de datos para encontrar información relevante.
- **Propósito**: El propósito de un retriever es filtrar grandes cantidades de datos y recuperar los documentos o entradas más relevantes que respondan a la consulta.
- **Uso**: Los retriever se utilizan en motores de búsqueda, sistemas de preguntas y respuestas y en cualquier lugar donde necesite extraer rápidamente piezas específicas de información de un gran conjunto de datos.

### Diferencias clave
- **Funcionalidad**: Los vector store se centran en almacenar y recuperar representaciones de datos numéricos, lo que los hace ideales para tareas que implican medir la similitud. Los retrievers, por su parte, están orientados a buscar en texto o datos para encontrar información relevante en función de una consulta.
- **Salida**: Los vector store devuelven vectores o puntuaciones en función de medidas de similitud, mientras que los retrievers proporcionan una lista de documentos o entradas de datos que se consideran relevantes para la consulta.
- **Función en los sistemas**: Los vector stores a menudo sirven como un componente de backend que respalda la función de los retrievers al proporcionar las representaciones de datos necesarias para la comparación. Los retrievers utilizan estos datos para realizar su función de búsqueda y obtención de información relevante.

En resumen, tanto los vector stores como los retrievers ayudan a gestionar y utilizar grandes conjuntos de datos, pero lo hacen de diferentes maneras. Los vector stores se centran en el almacenamiento y la recuperación de datos en formato numérico, mientras que los retrievers se centran en recuperar entradas de datos o textos relevantes en función de consultas específicas.

## Vector stores

Podemos utilizar muchos Vector Stores en nuestras aplicaciones LangChain. Aquí utilizaremos un vector stores Chorma.

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)

#### similarity_search()
Imagina que tienes una gran caja llena de varios juguetes y estás buscando juguetes que sean similares a tu coche de juguete favorito. Puedes empezar por sacar juguetes que también sean coches, pero luego limitar la búsqueda para encontrar coches que sean del mismo color o tamaño que tu coche favorito.

En términos informáticos, la búsqueda por similitud funciona de forma similar. Implica buscar en una gran cantidad de datos (como todos esos juguetes) para encontrar elementos que sean similares a un elemento específico que te interesa. Puede ser texto, imágenes o cualquier tipo de datos.

Cuando utilizas la búsqueda por similitud en una aplicación LangChain, esto es lo que suele pasar:
1. **Representación**: convertir palabras u oraciones en formas numéricas (llamadas embeddings).
2. **Comparación**: una vez que todo se convierte en números, compara estos números para ver qué tan similares son. Esto es como medir la distancia entre dos puntos.
3. **Recuperación**: el retriever ordena estos elementos según su similitud con la consulta (lo que está buscando) y le muestra los resultados más similares.

La función similarity_search() devuelve documentos en función de su similitud con una consulta de cadena:

In [ ]:
vectorstore.similarity_search("John")

#### similarity_search_with_score()
Cuando hablamos de `similarity_search_with_score`, estamos viendo un proceso un poco más detallado que simplemente encontrar elementos similares. Aquí se explica cómo puede entenderlo:

1. **Entrada y representación**:
- Primero, tiene una consulta, que es lo que le interesa para encontrar elementos similares. Esto podría ser un fragmento de texto, como una pregunta o un tema.
- El sistema convierte esta consulta y todos los elementos potenciales que podrían ser similares (como documentos o fragmentos de texto) en una forma numérica que representa sus significados. Esto generalmente se hace utilizando modelos que producen embeddings.

2. **Puntuación de similitudes**:
- Una vez que todo se convierte en estas embeddings numéricas, el sistema calcula la "distancia" entre el embedding de su consulta y las embeddings de otros elementos. Las distancias más cercanas significan que son más similares.
- El sistema utiliza una puntuación de similitud para cuantificar qué tan cerca o lejos está cada elemento de tu consulta. Esta puntuación suele estar entre 0 y 1, donde 1 significa extremadamente similar y 0 significa nada similar.

3. **Clasificación y recuperación**:
- En función de estas puntuaciones, el sistema clasifica todos los elementos desde el más similar al menos similar.
- A continuación, te presenta una lista de elementos, cada uno con una puntuación de similitud que muestra qué tan cerca está de coincidir con tu consulta.

Al utilizar herramientas de búsqueda de similitudes como las que se analizan en el curso LangChain, estas herramientas suelen convertir el texto en formas numéricas, o vectores, para medir qué tan similares son entre sí. Sin embargo, la forma en que se almacenan y comparan estos vectores puede diferir según la herramienta o el proveedor que utilices; cada uno puede tener su propio método para puntuar las similitudes.

A diferencia de otras herramientas que otorgan una puntuación de similitud en la que un número más alto significa más similitud, Chroma hace lo contrario. Utiliza una métrica de distancia para puntuar. En este caso:
- **Una distancia menor significa más similitud**: si la puntuación de distancia es cercana a 0, sugiere que los elementos son muy similares.
- **Una distancia mayor significa menos similitud**: si la puntuación de distancia es mayor, sugiere que los elementos son bastante diferentes.

Por lo tanto, en términos simples, cuando uses el vector store de Chroma para la búsqueda de similitudes, recuerda que estás buscando números (o distancias) más pequeños para encontrar más elementos similares, ya que estas puntuaciones varían inversamente con la similitud: ¡cuanto más pequeño, mejor!

In [ ]:
vectorstore.similarity_search_with_score("John")

## Retrievers

* Podemos crear un retriever manualmente, pero esta no es la opción que utilizaremos con más frecuencia. Una vez que elijamos qué método deseamos utilizar para recuperar documentos, podemos crear un retriever utilizando RunnableLambda. El código siguiente creará un retriever en torno al método similarity_search:

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result

retriever.batch(["John", "Robert"])

* La mayoría de las veces usaremos la función .as_retriever() para crear un Retriever usando el almacén vectorial:

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["John", "Robert"])

#### Los Retrievers son ejecutables
Los objetos LangChain VectorStore no son Runnables y, por lo tanto, no se pueden integrar de inmediato en LCEL. Por el contrario, los retrievers de LangChain si son ejecutables.
* Vea cómo usamos un retriever dentro de una cadena LCEL:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Responde esta pregunta, usando el contexto indicado.

{question}

Contexto:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

chain = {
    "context": retriever,
    "question": RunnablePassthrough()} | prompt | chatModel

In [ ]:
response = chain.invoke("Háblame sobre Jackie")

print(response.content)